In [1]:
import sys
import sqlite3
import math
import tqdm
import itertools

import pandas as pd

conn = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_chem_fix.sqlite')
c = conn.cursor()

# List of tables in the db
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in c.fetchall():
    print (table)

('stitch_chem',)
('stitch_protchem_man',)
('string_protlink_man',)
('chem_freq',)
('chem_uniq',)
('chem_hubs_orig',)
('string_protlink_man905',)
('chem_hubs',)
('string_prots_910',)
('stitch_chem40',)
('chem_hubs_4_10',)
('cid_prot_rank',)
('spearman_table',)
('akshai_chem_hubs',)


In [2]:
c.execute("pragma table_info(spearman_table);")
#c.execute("select * from stitch_protchem_man limit 1;")
for entry in c.fetchall():
    print (entry)

(0, 'cid1', 'int(10)', 1, None, 0)
(1, 'cid2', 'int(10)', 1, None, 0)
(2, 'spearman_number', 'float', 1, None, 0)


In [24]:
cid_output_file = open("cid_cluster_qmap_chemical.txt","r").readlines()
cluster_actual = {}
for entry in cid_output_file:
    cluster_actual[int(entry.split()[0])] = int(entry.split()[1])
print (len(cluster_actual))

18


In [535]:
input_file = open("distance_matrix_qmap_chemicals_out.csv","r").readlines()
all_cid = []
entire_data_dict = {}
loop = tqdm.tqdm(enumerate(input_file), total=len(input_file),leave=False)
for i,entry in loop:
    if i != 0:
        header = int(entry.split(",")[0].strip('"'))
        all_cid.append(header)
        entire_data_dict[header] = [float(distance) for i,distance in enumerate(entry.split(",")) if i != 0]
    

In [536]:
def get_as_df(beginning,end,all_cid,entire_data_dict):
    small_list = all_cid[beginning:end]
    
    distance_matrix = {}
    for entry in small_list:
        distance_matrix[entry] = entire_data_dict[entry]
        
    distancd_df = pd.DataFrame(distance_matrix)
    distancd_df.columns = small_list
    return (distancd_df,small_list)

In [569]:
distancd_df,small_list = get_as_df(0,18,all_cid,entire_data_dict)
distancd_df

,5757,448537,2733525,5035,104741,5280961,5281707,445154,6623,1752,3026,5281576,3224,3476,77999,2244,3672,3032
0,0.000000,0.134263,0.309543,0.360175,0.280422,0.452440,0.356542,0.556355,0.000000,0.185351,0.791209,0.110810,0.628638,0.585925,0.529451,0.659751,0.653818,0.469112
1,0.134263,0.000000,0.318191,0.364986,0.324370,0.438777,0.316479,0.574261,0.196250,0.090393,0.850384,0.216913,0.635764,0.602302,0.543723,0.648999,0.658973,0.526126
2,0.309543,0.318191,0.000000,0.285942,0.082678,0.430901,0.355195,0.428460,0.351218,0.373767,0.911438,0.356268,0.497791,0.680673,0.557274,0.646825,0.666540,0.574479
3,0.360175,0.364986,0.285942,0.000000,0.324657,0.256351,0.321466,0.458851,0.388349,0.432479,0.955715,0.441422,0.207683,0.587882,0.534150,0.679796,0.698558,0.644328
4,0.280422,0.324370,0.082678,0.324657,0.000000,0.403555,0.367090,0.394647,0.271416,0.407118,0.865484,0.418431,0.497230,0.538961,0.470493,0.579073,0.580861,0.555303
5,0.452440,0.438777,0.430901,0.256351,0.403555,0.000000,0.343552,0.416376,0.487032,0.529647,0.983062,0.592497,0.418501,0.649672,0.600864,0.732455,0.764417,0.628807
6,0.356542,0.316479,0.355195,0.321466,0.367090,0.343552,0.000000,0.654687,0.379259,0.405560,0.923326,0.475087,0.518727,0.636410,0.615474,0.635210,0.677706,0.584627
7,0.556355,0.574261,0.428460,0.458851,0.394647,0.416376,0.654687,0.000000,0.574058,0.663686,0.984963,0.640001,0.580686,0.729166,0.636164,0.753153,0.756744,0.707395
8,0.000000,0.196250,0.351218,0.388349,0.271416,0.487032,0.379259,0.574058,0.000000,0.288880,0.871782,0.264431,0.685400,0.688907,0.573889,0.715861,0.718596,0.615852
9,0.185351,0.090393,0.373767,0.432479,0.407118,0.529647,0.405560,0.663686,0.288880,0.000000,0.891926,0.185968,0.678513,0.639362,0.616477,0.755846,0.746967,0.482024


In [578]:
#for col in distancd_df.columns: 
#    print(col)
distancd_df.columns[0]

3032

In [ ]:
{0: [5280961,104741,5281576,5035,5281707,3476,2733525,1752,448537,5757,6623],
 1: [3026],
 2: [3224, 445154, 77999],
 3: [3672, 2244, 3032]}

In [739]:
batch_size = 2
number_of_batches = math.ceil(len(all_cid)/batch_size)

In [769]:
%%time
min_dist = 0.6
cids = all_cid

clusters = {}
cluster_leaf_dist = {}
check_list = set()
cluster_output = {}

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 37.2 µs


In [770]:
for i in range(number_of_batches):
    
    if i + 1 == number_of_batches:
        beginning = i * batch_size
        end = len(cids)
    else:
        beginning = i * batch_size
        end = (i + 1) * batch_size
         
    distancd_df,small_list = get_as_df(beginning,end,all_cid,entire_data_dict)
    
    df = distancd_df
    
    total_data_points = len(df)
    
    
    # For each datapoint (i.e., all the cids) find the distance with the cid in batch 
    # Then choose the one with distance less than the min_distance given
    # Then use that to make a dict of clusters of format
    # {all_cid1 : [batch_cids1],all_cid2: [batch_cids2]}
    # batch_cids1 and batch_cids2 etc are cids with length less than min distance
    # And also cluster_leaf_dist with format
    # {all_cid1 : [batch_cids1_distance],all_cid2: [batch_cids2_distance]}
    
    loop = tqdm.tqdm(range(total_data_points), total=total_data_points,leave=False)
    
    for j in loop:
        do_cluster = True
        
        if cids[j]  in check_list:
            do_cluster = False
            
        if do_cluster:
            df1 = df.iloc[j][df.iloc[j] < min_dist]
            cluster_leaf_dist[cids[j]] = list(df1)#[ distance if distance <= (min_dist * 0.5) else 0 for distance in list(df1)]
            #print (cluster_leaf_dist[cids[j]])
            clusters[cids[j]] = list(map(int,df1.index))
            
            if len(check_list) != (batch_size * (i + 1)):
                a = list(clusters.values())[-1]
                check_list.update(a)
                
        loop.set_description("Batch " + str(i+1) + "/" + str(number_of_batches))
    
    
    
    # Find number of occurance of each label in different clusters
    # For each {all_cid1 : [batch_cids1],all_cid2: [batch_cids2]}
    # {batch_cids1_1 : occurance,batch_cids1_2:occurance ...}
    
    label_dict = {entry : 0 for entry in cids[0:end]}
    for label_list in list(clusters.values()):
        for entry in label_list:
            label_dict[entry] += 1
    
    # For each {batch_cids1_1 : occurance,batch_cids1_2:occurance ...}
    # if occurance > 1
    # batch_cids1_1
    loop = tqdm.tqdm(label_dict, total=len(label_dict),leave=False)      
    # Find the one with more than one occurance and remove from the clusters
    for leaf in loop:
        if label_dict[leaf] > 1:
            leaf_index = small_list.index(leaf)
            root_leaf_distance = {}
            root_list = []
            reached_limit = False
            
            root_leaf_same = False
            # Find the distance for the leaf in each root/cluster centers
            i = 0
            for root in clusters:
                if len(root_leaf_distance) == label_dict[leaf]:
                    reached_limit = True
                    break
                if not reached_limit:
                    if leaf in clusters[root]:
                        #root_index = cids.index(root)
                        root_list.append(root)
                        root_leaf_distance[root] = cluster_leaf_dist[root][clusters[root].index(leaf)] + (0.0000001*i) #m.iloc[root_index,leaf_index]
                        if root == leaf:
                            root_leaf_distance[root] = cluster_leaf_dist[root][clusters[root].index(leaf)]
                            root_leaf_same = True
                            root_leaf_cid = root
                        i += 1
                                
            assert (len(root_leaf_distance) == label_dict[leaf])
            
            root_leaf_distance_values = list(root_leaf_distance.values())
            root_leaf_distance_keys = list(root_leaf_distance.keys())
            
            if not root_leaf_same:
                # Find the root with least distance
                min_value_index = root_leaf_distance_values.index(min(root_leaf_distance_values))
                belongs_to_root = root_list[min_value_index]

                # Remove the element from the cluster for the non-root
                root_list.remove(belongs_to_root)
            
            else:
                # Sort distance to find the second least distance if leaf == root in the above loop
                # This is to avoid self falling into the cluster with len 0
                sorted_distance = sorted(root_leaf_distance_values)
                
                # Remove the element from the cluster for the non-root
                second_minimum = sorted_distance[1]
                #print (root_leaf_cid,root_list[root_leaf_distance_values.index(first_minimum)],root_list[root_leaf_distance_values.index(second_minimum)])
                #if root_list[root_leaf_distance_values.index(first_minimum)] == root_leaf_cid:
                #    second_min_distance = second_minimum
                #else:
                #    second_min_distance = first_minimum
                second_min_distance = second_minimum
                if second_min_distance <= min_dist:
                    min_value_index = root_leaf_distance_values.index(second_min_distance)
                    belongs_to_root = root_list[min_value_index]
                else:
                    min_value_index = root_leaf_distance_values.index(min(root_leaf_distance_values))
                    belongs_to_root = root_list[min_value_index]
                root_list.remove(belongs_to_root)
                
            for entry in root_list:
                #print (leaf)
                clusters[entry].remove(leaf)
        loop.set_description("Finding and sorting the data to clusters Batch " + str(i+1) + "/" + str(number_of_batches))
        
    #print (cluster_output)
    for entry in clusters:
        if entry not in cluster_output:
            cluster_output[entry] = []
            cluster_output[entry].extend(clusters[entry])
        else:
            for entry1 in clusters[entry]:
                cluster_output[entry].append(entry1)
                cluster_output[entry] = list(set(cluster_output[entry]))
            
    #print (cluster_output)
    #for entry in clusters:
    #    if len(clusters[entry]) == 0:
    #        del clusters[entry]
    #print (len(clusters))
    #for entry in clusters:
    #    if len(clusters[entry]) > 0:
    #        print (entry,len(clusters[entry]))

In [771]:
print (cluster_output)

{5757: [448537], 2733525: [], 5035: [5280961, 5035, 5281707, 3476, 3224], 104741: [2733525], 5280961: [445154], 5281707: [], 445154: [], 6623: [5281576, 104741, 5757], 1752: [], 3026: [3026], 5281576: [1752, 6623], 3224: [], 3476: [77999], 77999: [], 2244: [], 3672: [3032], 3032: [3672, 2244]}


In [772]:
final_output = {entry: [] for entry in cluster_output}
for entry in cluster_output:
    if len(cluster_output[entry]) > 0:
        for entry1 in cluster_output[entry]:
            if entry1 in final_output:
                if len(final_output[entry1]) > 0:
                    final_output[entry1].append(entry1)
                    final_output[entry1].extend(cluster_output[entry1])
                    final_output[entry1] = list(set(final_output[entry1]))
                    cluster_output[entry1] = []
                else:
                    final_output[entry].append(entry1)
                    final_output[entry].extend(cluster_output[entry1])
                    final_output[entry] = list(set(final_output[entry]))
                    cluster_output[entry1] = []
            else:
                final_output[entry].append(entry1)
                try:
                    final_output[entry].extend(cluster_output[entry1])
                    final_output[entry] = list(set(final_output[entry]))
                except:
                    final_output[entry] = list(set(final_output[entry]))

In [778]:
final_output

{5757: [448537, 5757],
 2733525: [],
 5035: [5280961, 445154, 5035, 5281707, 77999, 3476, 3224],
 104741: [104741, 2733525],
 5280961: [],
 5281707: [],
 445154: [],
 6623: [5281576, 6623, 1752],
 1752: [],
 3026: [3026],
 5281576: [],
 3224: [],
 3476: [],
 77999: [],
 2244: [],
 3672: [3032, 3672, 2244],
 3032: []}

In [774]:
#final_output

In [775]:
cluster_dict = {}
check_list = []
i = 0
for entry in final_output:
    if len(final_output[entry]) > 0:
        cluster_dict[i] = final_output[entry]
        for entry1 in final_output[entry]:
            check_list.append(entry1)
        i += 1

In [776]:
len(set(check_list)),len(cluster_dict)

(18, 6)

In [777]:
cluster_dict

{0: [448537, 5757],
 1: [5280961, 445154, 5035, 5281707, 77999, 3476, 3224],
 2: [104741, 2733525],
 3: [5281576, 6623, 1752],
 4: [3026],
 5: [3032, 3672, 2244]}

In [749]:
#for entry in cluster_dict:
#    if len(cluster_dict[entry]) > 1:
#        print (entry,cluster_dict[entry])

In [750]:
cluster_leader = {}
check_list = []
for entry in cluster_dict:
    #if len(cluster_dict[entry]) > 0:
    for entry1 in cluster_dict[entry]:
        cluster_leader[entry1] = entry
        check_list.append(entry1)
    #else:
    #    check_list.append(entry)
    #    cluster_leader[entry] = entry

In [751]:
len(set(check_list)),len(cluster_dict),len(cluster_leader),len(cluster_actual)

(18, 15, 18, 18)

In [752]:
len(cluster_leader)

18

In [754]:
cid_output_file = open("cid_cluster_qmap_chemical.txt","r").readlines()
cluster_actual = {}
for entry in cid_output_file:
    cluster_actual[int(entry.split()[0])] = int(entry.split()[1])
print (len(cluster_actual))

18


In [755]:
# Comaprison of K-mean and K-mean distance matrix
# Given two dict, it compares the overlap (dict of cid and cluster)
# Perfect overlap will have each label having list of one high number and rest all zero
def find_cluster_overlap(dict1,dict2):
    
    dict_1_clusters = {int(dict1[entry]) : [] for entry in dict1.keys()}
    for entry in dict1.keys():
        dict_1_clusters[int(dict1[entry])].append(int(entry))
    
    dict_2_clusters = {int(dict2[entry]) : [] for entry in dict2.keys()}
    for entry in dict2.keys():
        dict_2_clusters[int(dict2[entry])].append(int(entry))
        
    all_label_clustered = {int(dict1[entry]) : [0 for entry in dict_2_clusters.keys()] for entry in dict1.keys()}
    
    for label1 in dict_1_clusters:
        for label2 in dict_2_clusters:
            for entry in dict_1_clusters[label1]:
                if entry in dict_2_clusters[label2]:
                    all_label_clustered[label1][label2] += 1
                    
    total = 0
    for entry in all_label_clustered:
        total += sum(all_label_clustered[entry])
    return (total,all_label_clustered)
total_clusters,label_overlap = find_cluster_overlap(cluster_actual,cluster_leader)
#18 {4: [5, 1, 0, 0, 1], 0: [0, 4, 0, 0, 1], 3: [0, 0, 0, 1, 0], 2: [0, 0, 0, 0, 2], 1: [0, 0, 3, 0, 0]}

In [756]:
total_clusters
for entry in label_overlap:
    print (entry,label_overlap[entry])

2 [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
14 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
13 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
12 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
8 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
5 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
3 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
6 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [757]:
count = 0
for entry in label_overlap:
    multi_list = []
    for entry1 in label_overlap[entry]:
        if entry1 >= 1:
            multi_list.append(entry1)
    if len(multi_list) > 1:
        print (entry,multi_list)
        count += 1
print (count)

0 [1, 1]
1


In [758]:
def get_other_max(label_overlap,actual_label,chosen_max,index):
    for entry in label_overlap:
        if entry!= actual_label:
            current_max = label_overlap[entry][index]
            if current_max > chosen_max:
                found_new_max = True
                return found_new_max
    return False

def get_accuracy(label_overlap):
    
    finished_clusters = []
    each_row_accuracy = {}
    for actual_label in label_overlap:
        count_dict = {} # Count of clusters more than 0 and their index in the list for each label in the actual dict
        for i,count in enumerate(label_overlap[actual_label]):
            if count > 0:
                count_dict[i] = count
        
        # Find the clusters which is mostly in the label, then find whether it is accumulated maximum in the label, if yes find accuracy else
        # find the next max and do the procedure
        calculated_accuracy = False
        count_dict = {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1])}
        for i in count_dict:
            if i not in finished_clusters:
                chosen_max = count_dict[i]
                if not get_other_max(label_overlap,actual_label,chosen_max,i):
                    row_accuracy = chosen_max/(sum(count_dict.values()))
                    calculated_accuracy = True
                    finished_clusters.append(i)
                    break
        
        # If accuracy was never found with the max element from the above code, use the max value in the list to get the accuracy
        if not calculated_accuracy:
            for i in count_dict:
                if i not in finished_clusters:
                    chosen_max = count_dict[i]
                    row_accuracy = chosen_max/(sum(count_dict.values()))
                    calculated_accuracy = True
                    finished_clusters.append(i)
                    break
        
        if not calculated_accuracy:
            row_accuracy = 0
        
        each_row_accuracy[actual_label] = row_accuracy
    row_accuracy = each_row_accuracy.values()
    print (sum(row_accuracy)/len(row_accuracy))

In [759]:
get_accuracy(label_overlap)
#0.9454352646321534
#0.9301186220463726
#0.9296067787115542

0.9666666666666667


In [ ]:
5757: 4,
448537: 4,
2733525: 4,
104741: 4, 
6623: 4,
1752: 4,
5281576: 4,
    
5035: 0,
5280961: 0,
5281707: 0,
3224: 0,
    
445154: 5,

3026: 3,

3476: 2,
77999: 2,
    
2244: 1,
3672: 1,
3032: 1